# Additive Backend

> The additive backend can be used to obtain pathlengths and group delays.

In [1]:
from functools import partial

import jax
import jax.example_libraries.optimizers as opt
import jax.numpy as jnp
import matplotlib.pyplot as plt  # plotting
import sax

## Parametrized Models

In [2]:
def coupler(length=50.0) -> sax.SDict:
    sdict = {
        ("in0", "out0"): length,
        ("in0", "out1"): length,
        ("in1", "out0"): length,
        ("in1", "out1"): length,
    }
    return sax.reciprocal(sdict)

In [3]:
def waveguide(length=100.0) -> sax.SDict:
    sdict = {
        ("in0", "out0"): length,
    }
    return sax.reciprocal(sdict)

## Circuit with additive backend

In [4]:
mzi, _ = sax.circuit(
    netlist={
        "instances": {
            "lft": coupler,
            "top": partial(waveguide, length=500),
            "btm": partial(waveguide, length=100),
            "rgt": coupler,
        },
        "connections": {
            "lft,out0": "btm,in0",
            "btm,out0": "rgt,in0",
            "lft,out1": "top,in0",
            "top,out0": "rgt,in1",
        },
        "ports": {
            "in0": "lft,in0",
            "in1": "lft,in1",
            "out0": "rgt,out0",
            "out1": "rgt,out1",
        },
    },
    backend="additive",
)

In [5]:
mzi()

{('in0', 'in0'): [Array([0.], dtype=float64)],
 ('in0', 'out0'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('in0', 'out1'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('in1', 'in1'): [Array([0.], dtype=float64)],
 ('in1', 'out0'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('in1', 'out1'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('out0', 'in0'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('out0', 'in1'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('out0', 'out0'): [Array([0.], dtype=float64)],
 ('out1', 'in0'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('out1', 'in1'): [Array([200.], dtype=float64), Array([600.], dtype=float64)],
 ('out1', 'out1'): [Array([0.], dtype=float64)]}